<a href="https://colab.research.google.com/github/mhummel06/WCT-Competition-Leaderboard/blob/main/WCT_Ladder_Elo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import math

from google.colab import auth
auth.authenticate_user()

import gspread
#from oauth2client.client import GoogleCredentials
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

#tagstat = gc.open('Title of Google Sheet')
tagstat = gc.open('WCT Stat Elo ALL')

ratings = tagstat.worksheet('Ratings')
tagData = tagstat.worksheet('Data')
archive = tagstat.worksheet('Archive')

#Global Constants for now
_K = 42 #Rating Risk Maximum
_WEIGHT = 1.0 #Event Weight , affects _K
_S = 256 #Scale Factor representing relationship of Player Skill on expected results
_Adv = 124 #Estimated Advantage of Chaser role
_StartElo = 1000.0 #Starting Elo

#Expected Outcome Calculation (Chaser)
def Exp_C(R1, R2, Adv, S):
  return (1.0/(1+10**((float(R2)-float(R1)-Adv)/S)))

#Expected Outcome Calculation (Evader)
def Exp_E(Exp_C_result):
  return 1.0 - Exp_C_result

#Update Rating - returns new rating
#R0 = Current Rating | RA = actual evade result, 1 or 0 for success or fail | RE = Expected via Exp_C or Exp_E func | K = _K
def newRating(R0, RA, RE, K):
  return (R0 + (K*(RA-RE)))

#Each Row from Ratings consists of a
#[0]Player Name, #[1]Current Rating, #[2]Chase winloss,
#[3]Evade winloss, #[4]Chase role Win, #[5]Evade role Win,
#[6|7]losses for each, and [8]N of Matches total
listing = ratings.get_all_values()
for i in listing[1:]: #get_all_values pulls all data into an array as str (string) types, so we need to cast the numbers into floats and ints
  i[1] = float(i[1])
  i[2] = float(i[2])
  i[3] = float(i[3])
  i[4] = int(i[4])
  i[5] = int(i[5])
  i[6] = int(i[6])
  i[7] = int(i[7])
  i[8] = int(i[8])
#print (listing)

#Each Row from Data consists of a match that must be fielded and archived, then each player's ratings updated
# Chaser, Evader, Time
rows = tagData.get_all_values()
rows.pop(0)
#print (rows)

#arch is full archive as list
#arch = archive.get_all_values()
#print (arch)

#create counter for consecutive evasions that will tick if previous evader prevEvader == current evader
consecEvade = 0
prevEvader = None
"""
#data collection for rate of multi-evade
One = 0
Two = 0
Three = 0
Four = 0
Five = 0
Six = 0
Seven = 0
Eight = 0
"""
#for each row(Chase) find and update Chaser's Rating, and Evader's Rating based on results
for x in rows:
  Player_Chase = x[0]
  Player_Evade = x[1]
  duration = float(x[2])

  #match_ID = '=Row(a' + index(x) + ')'

  #Consecutive evasions tracker
  if Player_Evade == prevEvader:
    consecEvade += 1
##    print(prevEvader, consecEvade) #debug helper to see it counting
  else:
    consecEvade = 0
    prevEvader = x[1]
  """ #Counting the readable record's consecutive evasions
  if consecEvade == 1:
    One += 1
  elif consecEvade == 2:
    Two += 1
    One -= 1
  elif consecEvade == 3:
    Three += 1
    Two -= 1
  elif consecEvade == 4:
    Four += 1
    Three -= 1
  elif consecEvade == 5:
    Five += 1
    Four -= 1
  elif consecEvade == 6:
    Six += 1
    Five -= 1
  elif consecEvade == 7:
    Seven += 1
    Six -= 1
  elif consecEvade == 8:
    Eight += 1
    Seven -= 1
  """
  #Search for the Chaser on the leaderboard
  chaser_rec = None
  chaser_index = 0
  #chaser_rec = ratings.find(Player_Chase)

  #search for existing match on the leaderboard
  for i in listing[1:]:
    if i[0] == Player_Chase:
      chaser_rec = i
      chaser_index = listing.index(i)
  #if chaser_rec != None:
    #chaser_rec = ratings.row_values(chaser_rec.row) #get chaser record
  if chaser_rec == None: #create listing for new player (Chaser) if none was found
    chaser_rec = [Player_Chase, _StartElo, 0, 0, 0, 0, 0, 0, 0]
    listing.append(chaser_rec) #first append call
    chaser_index = listing.index(chaser_rec)

  #Search for the Evader on the Leaderboard
  evade_rec = None
  evade_index = 0
  #evade_rec = ratings.find(Player_Evade)
  for i in listing[1:]:
    if i[0] == Player_Evade:
      evade_rec = i
      evade_index = listing.index(i)
  #if evade_rec != None:
    #evade_rec = ratings.row_values(evade_rec.row) #get evader record
  #print(chaser_rec)
  if evade_rec == None: #create listing for new player (Evader) if none was found
    evade_rec = [Player_Evade, _StartElo, 0, 0, 0, 0, 0, 0, 0]
    listing.append(evade_rec)
    evade_index = listing.index(evade_rec)

  #Calculate expected odds based on ratings
  RE_Chase = Exp_C(chaser_rec[1],evade_rec[1],_WEIGHT * _Adv,_WEIGHT * _S)
  RE_Evade = Exp_E(RE_Chase)

  #Print check for E values
  #print("Chaser:", Player_Chase, "Evader:", Player_Evade, "Expected win Chance (Chaser):",RE_Chase, "Ratings:", chaser_rec[1], evade_rec[1])

  x.append(chaser_rec[1])
  x.append(evade_rec[1])

  rating_delta = 0
  if duration < 20.0: #tag was made
    #Adding in protective measures for multi-evasion, increasing predicted chance of tag, meaning worth a bit less
    if consecEvade > 0:
      RE_Chase *= 1 + (consecEvade * 0.08)
      if RE_Chase > 1.0:
        RE_Chase = 0.99
    RE_Evade = 1 - RE_Chase
    rating_delta = ((1.0 - RE_Chase) * (_WEIGHT * _K)) - (_WEIGHT * (math.log(duration/9) + 0.2))
    x[3] = rating_delta
    chaser_rec[4] += 1
    evade_rec[7] += 1
    chaser_rec[8] += 1
    evade_rec[8] += 1
    #Duration Curve added here - _WEIGHT matching % weight for event tier
    chaser_rec[1] = newRating(chaser_rec[1],1,RE_Chase,(_WEIGHT * _K)) - (_WEIGHT * (math.log(duration/9)) + 0.2)
    evade_rec[1] = newRating(evade_rec[1],0,RE_Evade,(_WEIGHT * _K)) + (_WEIGHT * (math.log(duration/9) + 0.2))
  elif duration == 20.0: #evader point
    if consecEvade > 1:
      RE_Chase *= 1 + ((consecEvade - 1) * 0.25)
      if RE_Chase > 1.0:
        RE_Chase = 0.99
    RE_Evade = 1 - RE_Chase
    rating_delta = (0.0 - RE_Chase) * (_WEIGHT * _K)
    x[3] = rating_delta
    chaser_rec[6] += 1
    evade_rec[5] += 1
    chaser_rec[8] += 1
    evade_rec[8] += 1
    chaser_rec[1] = newRating(chaser_rec[1],0,RE_Chase,(_WEIGHT * _K))
    evade_rec[1] = newRating(evade_rec[1],1,RE_Evade,(_WEIGHT * _K))

  #Ratings are updated, this match recorded

  #print("Points risked:", (1.0 - RE_Chase) * _K, "on evasion:", (0.0 - RE_Chase) * _K)

  #transfer match x to archive
  #arch.append(x)
  #print(x[0:4])
  #archive.append_row(x)

  #erases match from input data
  #tagData.delete_rows(rows.index(x))

  #Recalculate winloss
  chaser_rec[2] = chaser_rec[4] / (chaser_rec[4] + chaser_rec[6])
  evade_rec[3] = evade_rec[5] / (evade_rec[5] + evade_rec[7])

  listing[chaser_index] = chaser_rec
  listing[evade_index] = evade_rec

#final update to Ratings
ratings.update(listing)
#update archive
archive.append_rows(rows)
#import pandas as pd
#pd.DataFrame.from_records(rows)

#print(One, Two, Three, Four, Five, Six, Seven, Eight)

{'spreadsheetId': '1jg67wDzveq8v8Gp0LBpfZ225Llv2vB_QCRdTTRZo4s4',
 'tableRange': 'Archive!A1:F1',
 'updates': {'spreadsheetId': '1jg67wDzveq8v8Gp0LBpfZ225Llv2vB_QCRdTTRZo4s4',
  'updatedRange': 'Archive!A2:F3468',
  'updatedRows': 3467,
  'updatedColumns': 6,
  'updatedCells': 20802}}

In [ ]:
import gspread
!pip install --upgrade gspread

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 1.8 MB/s eta 0:00:00
  Attempting uninstall: gspread
    Found existing installation: gspread 3.4.2
    Uninstalling gspread-3.4.2:
      Successfully uninstalled gspread-3.4.2


In [ ]:
#Code for Archive Analysis (WIP)
import math

from google.colab import auth
auth.authenticate_user()

import gspread
#from oauth2client.client import GoogleCredentials
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

tagstat = gc.open('Card Stats')
#tagstat = gc.open('WCT Stat Elo ALL')

